In [1]:
import os
import nibabel as nib
import six, numpy as np
import SimpleITK as sitk
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
from scipy.ndimage import label
from scipy import ndimage
from nilearn.plotting import view_img, glass_brain, plot_anat, plot_epi

from ipywidgets import widgets, interact
import k3d


import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

1 - Ler o csv total: full_information.csv

In [2]:
def read_csv_dataset(csv_dataset_file):
    df = pd.read_csv(csv_dataset_file)
    return df

csv_dataset_file = 'full_information.csv'
df=read_csv_dataset(csv_dataset_file)

2 - Fazer o display do csv total: full_information.csv

In [4]:
#Lê todos os scans para uma estrutura nparray
def load_scans(df, path):
    #print(df.columns.values)
    
    todos_scans=[]
    n=0
    for index, row in df.iterrows():
        ficheiro=path+row['Path']+'/'+row['Scan']
        #print(ficheiro)
        scan = nib.load(ficheiro)
        s=np.asarray(scan)
        todos_scans.append(s)
        #print(scan.shape)
    #print(s)
    np_todos_scans = np.asarray(todos_scans)
    return np_todos_scans
    
        
dataPath = '../../data/nii/'
np_todos_scans=load_scans(df, dataPath ) 
print("np_todos_scans shape:", len(np_todos_scans))



FileNotFoundError: No such file or no access: '../../data/nii/CHUM-006/CT/image.nii.gz'

In [7]:
#Mostrar tudo iterativamente
def show_scans(patient,path):
    registo=df.loc[df['Patient'] == patient]
    registo_lista=registo.iloc[0]
    ficheiro_scan=path+registo_lista['Path']+'/'+registo_lista['Scan']
    ficheiro_mask=path+registo_lista['Path']+'/'+registo_lista['GTV Primary']
    #print("Scan: ", ficheiro_scan)
    #print("Mask: ", ficheiro_mask)
    scan = nib.load(ficheiro_scan)
    scan_np = scan.get_fdata().T
    mask = nib.load(ficheiro_mask)
    mask_np = mask.get_fdata().T
    #print(mask_np)
    print(scan_np.shape)
    print(mask_np.shape)
    #print(scan_np.shape[0])
    @interact
    def show_images(Slice=(0, scan_np.shape[0]-1)):
        fig, (img1, img2) = plt.subplots(1, 2)
        fig.suptitle('Horizontally stacked subplots')
        img1.imshow(scan_np[Slice], cmap='gray')
        img2.imshow(mask_np[Slice], cmap='gray')
    return True
 
    
dataPath = '../../data/nii/'
lista_de_casos= df['Patient'].tolist()

@interact
def show_patients(Patient=lista_de_casos):
    show_scans(Patient,dataPath)
    return True


interactive(children=(Dropdown(description='Patient', options=('CHUM-001', 'CHUM-002', 'CHUM-003', 'CHUM-004',…

## Flip Scans

In [21]:
a = pd.read_csv('full_information.csv')
dataPath = '../../data/Head-Neck/'

from PIL import Image
import numpy as np

#it is necessary to flip the CHUM-006, CHUM-008, CHUM-009 and CHUM-010 
#[5],[6],[7] and [8]

scanPath = dataPath + a['Path'][8]+ '/'+ a['Scan'][8]
maskPath = dataPath + a['Path'][8]+ '/'+ a['GTV Primary'][8]

  
image = sitk.ReadImage(scanPath)
img_array = sitk.GetArrayFromImage(image)
flipped = np.flip(img_array, axis=1)
img_flip=sitk.GetImageFromArray(flipped)
sitk.WriteImage(img_flip,'../../data/Head-Neck/CHUM-010/CT/image.nii.gz') 



"\nfor index in range(z):\n    im= img_array[index,:,:]\n  \n    plt.figure(figsize=(10,10))\n    plt.imshow(flipped[index,:,:], cmap='gray')\n"